In [ ]:
import json
import os

import riotwatcher
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("RIOT_API_KEY")
REGION = "EUW1"

lol_watcher = riotwatcher.LolWatcher(API_KEY)

In [ ]:
from tqdm import tqdm
# FILEPATH: /D:/Programmation/Python/Projets de ouf/ChampSelectPrediction/getmatches.ipynb
m = lol_watcher.league.masters_by_queue(REGION, "RANKED_SOLO_5x5")
gm = lol_watcher.league.grandmaster_by_queue(REGION, "RANKED_SOLO_5x5")
c = lol_watcher.league.challenger_by_queue(REGION, "RANKED_SOLO_5x5")
players_dict: dict = json.load(open("players.json", "r"))

with tqdm(total=len(m["entries"] + gm["entries"] + c["entries"])) as pbar:
    for player in m["entries"] + gm["entries"] + c["entries"]:
        if not player["summonerName"]:
            print("No summoner name")
            continue
        if player["summonerName"] in players_dict:
            print("Already in")
            continue
        summoner = lol_watcher.summoner.by_name(REGION, player["summonerName"])
        players_dict[player["summonerName"]] = summoner["puuid"]
        json.dump(players_dict, open("players.json", "w"))
        pbar.update(1)


## Find the last 5 games for each master grandmaster and challenger player

In [ ]:
from tqdm import tqdm

players_dict = json.load(open("players.json", "r"))
games_dict = json.load(open("games_ids.json", "r"))

with tqdm(total=len(players_dict)) as pbar:
    for player_name, player_puid in players_dict.items():
        if player_name in games_dict:
            print("Already in")
            pbar.update(1)
            continue

        matches = list(
            lol_watcher.match.matchlist_by_puuid(REGION, player_puid, count=5)
        )
        if not matches:
            print("No matches")
            pbar.update(1)
            continue
        games_dict[player_name] = matches
        json.dump(games_dict, open("games_ids.json", "w"))
        pbar.update(1)

print("Done")

In [26]:
g=lol_watcher.match.by_id("EUW1", "EUW1_6807392992")

In [30]:
g["info"]["participants"][0]["teamId"]


100

## Extract champions used in each match and the result of the match

In [31]:
import pandas as pd
from tqdm import tqdm
games_players_dict = json.load(open("games_ids.json", "r"))
games = pd.read_csv("soloq_games_data.csv")
games.set_index("gameID")

with tqdm(total=len(games_players_dict)) as pbar:
    for player_name, games_ids in games_players_dict.items():
        for game_id in games_ids:
            if game_id in games["gameID"]:
                pbar.update(1)
                continue
            game_data = lol_watcher.match.by_id("EUW1", game_id)["info"]
            champion_names = [game_data["participants"]
                              [i]["championName"] for i in range(10)]
            result = "Red" if game_data["participants"][0]["teamId"] == 100 else "Blue"
            games.loc[game_id] = [
                game_id, game_data["gameVersion"], "Red", *champion_names]

            # save the data
            games.to_csv("soloq_games_data.csv", index=False)
            pbar.update(1)

  0%|          | 0/1885 [00:00<?, ?it/s]

  5%|▌         | 100/1885 [00:13<03:17,  9.06it/s]

In [21]:
import pandas as pd

df = pd.DataFrame({
    "gameid": [0, 1, 2],
    "patch": [1, 2, 3]
})

df.set_index("gameid")

df.loc[3] = [3,4]

df.to_csv("test.csv", index=False)